In [1]:
from math import pi
import math
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# import Qiskit
from qiskit import Aer,  execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

from qiskit.aqua.components.feature_maps.raw_feature_vector import RawFeatureVector

import warnings
import itertools
warnings.filterwarnings("ignore", category=DeprecationWarning) 
plt.style.use('seaborn-whitegrid')

In [2]:
# UTILS FOR COMPUTING EFFICIENCIES
def makeitinteger(centers):
    intcenters=[int(centers[0])]
    for i in range(1,len(centers)):
        integer=int(centers[i])
        intcenters=np.append(intcenters,int(integer))
    return intcenters
def compute_efficiency(o_centers,intcenters):
    eff=0
    #intcenters=makeitinteger(centers)
    for i in range(len(intcenters)):
        if (o_centers[i]==intcenters[i]):
            eff+=1
    return (eff/len(intcenters))
def changing_index(a,b,intcenters):
    intcentersaux=np.where(intcenters==a, b,intcenters)
    intcenters=np.where(intcenters==b,a,intcentersaux)
    return intcenters
def final_efficiency(o_centers,centers,k):
    effmax=0
    intcenters=makeitinteger(centers)
    goodcenters=intcenters.copy()
    for i in range(0,k-1):
        for j in range(0,k):#1
            intcentersaux=changing_index(i,j,intcenters)
            
            eff=compute_efficiency(o_centers,intcentersaux)
            
            if(eff>effmax):
                effmax=eff
                goodcenters=intcentersaux
                intcenters=goodcenters
    return (effmax,goodcenters)
def final_efficiency_converged(o_centers,centers):
    k=max(o_centers)+1
    intcenters=makeitinteger(centers)
    stop=0
    ef,intcenters=final_efficiency(o_centers,intcenters,k)# we are changing intcenters 
    while (stop==0):
        efprime,intcentersprime=final_efficiency(o_centers,intcenters,k)
        if (efprime==ef):
            stop=1
        else:
            ef=efprime
            intcenters=intcentersprime
    return (ef,intcenters)


In [3]:
# OBTAINING QUANTUM MINIMUM
def get_quantum_maximum(array,shots):
    backend = Aer.get_backend('qasm_simulator') 
    X=np.array(array)
    fvc = RawFeatureVector(len(X))
    q1 = QuantumRegister(fvc.num_qubits)
    minimum_circuit = fvc.construct_circuit(X, qr=q1)
    minimum_circuit.measure_all()
    result = execute(minimum_circuit, backend, shots=shots).result()
    counts=result.get_counts()
    max_value = max(counts.values())
    max_values_counts = [x[0] for x in counts.items() if x[1] == max_value]
    #result=counts.most_frequent()
    result=max_values_counts[0]
    
    index_minimum=int(result,2)
    return index_minimum
def get_quantum_minimum(array,shots,a_power):
    array_power=np.array(array)**a_power
    array_inv=1/array_power
    index_min=get_quantum_maximum(array_inv,shots)

    return index_min

In [4]:
# PROCESS AND TRANSFORMATION OF DATA

def process_together(points):
    maximum=points.max()
    minimum=points.min()
    points_std=(points-minimum)/(maximum-minimum)
    points_scaled=points_std*(10-1)+1
    return (points_scaled,maximum,minimum)

def transform_all_to_angular_4(data_momentum_vector):
    data_angular_vector=data_momentum_vector.copy()
    for i in range(len(data_momentum_vector)):
        data_angular_vector[i]=transform_to_angular_4(data_momentum_vector[i])
    
    return(data_angular_vector)
    
def transform_to_angular_4(data_momentum):
    data_angular=data_momentum.copy()
    data_angular[0]=(data_momentum[0]**2+data_momentum[1]**2)**0.5
    data_angular[1]=0.5*np.log(abs((data_momentum[3]+data_momentum[2])/
                            (data_momentum[3]-data_momentum[2])))#abs
    data_angular[2]=math.atan(data_momentum[1]/data_momentum[0])
    if(data_momentum[0]<0):
        data_angular[2]+=pi
    data_angular[3]=(data_momentum[3]**2-data_momentum[2]**2)**0.5#m_T
    #data_angular[2]+=2*pi
    return(data_angular)

def transform_all_to_momentum_4(data_angular_vector):
    data_momentum_vector=data_angular_vector.copy()
    for i in range(len(data_momentum_vector)):
        data_momentum_vector[i]=transform_to_momentum_4(data_angular_vector[i])
    return(data_momentum_vector)
def transform_to_momentum_4(data_angular):
    data_momentum=data_angular.copy()
    data_momentum[0]=data_angular[0]*math.cos(data_angular[2])
    data_momentum[1]=data_angular[0]*math.sin(data_angular[2])
    data_momentum[2]=data_angular[3]*math.sinh(data_angular[1])
    data_momentum[3]=data_angular[3]*math.cosh(data_angular[1])
    return data_momentum

In [5]:
# FUNCTIONS TO FIND THE MINIMUM AMONG THE DISTANCES
def find_minimum_cut(data,a,R,ptmin):
    #data_aux=data.copy()
    rapidity_plus_phi=data[:,1:3]
    rapidity_plus_phi_processed,max_processed,min_processed=process_together(rapidity_plus_phi)
    #print(rapidity_plus_phi_processed)
    min_dis=10**10
    lista=[]
    for i in range(len(data)):
        piB=data[i,0]**a
        min_dis_j=10**10
        #print("piB",i,piB)
        #lista.append([piB,i,len(data)+5])
        for j in range(i+1,len(data)):
            pij=kt_distance(data[i],data[j],rapidity_plus_phi_processed[i],
                                rapidity_plus_phi_processed[j],max_processed,min_processed,
                                a,R)
                #print("pij",i,j,pij)
            #lista.append([pij,i,j])
            if(pij<min_dis_j):
                min_dis_j=pij
                jlgood=j
        if(min_dis_j<piB or data[i,0]<ptmin):
            if(min_dis_j<min_dis):
                min_dis=min_dis_j
                igood=i
                jgood=jlgood
        else:
            if(piB<=min_dis ):
                min_dis=piB
                igood=i
                jgood=len(data)+5#the minimum distance is wrt to the beam
    #print(lista)
    return(min_dis,igood,jgood)

def find_minimumN2_quantum_cut(data,a,R,ptcut,shots,a_power):
    rapidity_plus_phi=data[:,1:3]
    rapidity_plus_phi_processed,max_processed,min_processed=process_together(rapidity_plus_phi)
    lista=[]
    
    for i in range(len(data)):
        piB=data[i,0]**a
        if (data[i,0]>ptcut):
            lista.append([piB,i,len(data)+5])
        else:
            if(len(data)==2):
                lista.append([piB+999999999999,i,len(data)+5])
        listaj=[]
        for j in range(i+1,len(data)):
            pij=kt_distance(data[i],data[j],rapidity_plus_phi_processed[i],
                                rapidity_plus_phi_processed[j],max_processed,min_processed,
                                a,R)
            listaj.append([pij,i,j])
            
        arrayj=np.array(listaj)
        if(len(arrayj)>1):
            indexj=get_quantum_minimum(arrayj[:,0],shots,a_power)
            min_disj=arrayj[indexj,0]
            igoodj=int(arrayj[indexj,1])
            jgoodj=int(arrayj[indexj,2])
            lista.append([min_disj,igoodj,jgoodj])
        else:
            if(len(arrayj)==1):
                indexj=0
                min_disj=arrayj[indexj,0]
                igoodj=int(arrayj[indexj,1])
                jgoodj=int(arrayj[indexj,2])
                lista.append([min_disj,igoodj,jgoodj])
    array=np.array(lista,dtype=object)
    index=get_quantum_minimum(array[:,0],shots,a_power)
    min_dis=array[index,0]
    #print(array[index,1],array)
    igood= int(array[index,1]) 
    jgood=int(array[index,2])
    #print(lista)
    return(min_dis,igood,jgood)

In [6]:
# FUNCTIONS FOR RUNNING KT ALGORITHM

def kt_distance(p1_angular,p2_angular,rapidity_plus_phi1,
                                    rapidity_plus_phi2,max_processed,min_processed,a,R):

    deltaR2=deltaR2_classical(p1_angular[1:3],p2_angular[1:3])
    
    min_pts=min(p1_angular[0]**a,p2_angular[0]**a)
    dij=min_pts*deltaR2/(R**2)
    return dij
def deltaR2_classical(p1_angles,p2_angles):
    return((p1_angles[0]-p2_angles[0])**2+(p1_angles[1]-p2_angles[1])**2)



def merge_or_remove(indexes,indexes_jets,final_jets,data_angular,
                    data_momentum,a,R,qsearch,ptcut,shots,a_power):

    if(qsearch):
        min_dis,i,j=find_minimumN2_quantum_cut(data_angular,a,R,ptcut,shots,a_power)#quantum
    else:
        min_dis,i,j=find_minimum_cut(data_angular,a,R,ptcut)
    

    final_jets_copy=final_jets.copy()
    if(j==(len(data_angular)+5)):
        #print("remove:",i,min_dis,data_angular[i,0])
        indexes_jets,indexes,final_jets_copy,data_angular,data_momentum=remove(indexes,indexes_jets,final_jets_copy,data_angular,data_momentum,i)
        #print(indexes_jets)
    else:
        #print("merge:",i,j,min_dis)
        indexes,data_angular,data_momentum=merge(indexes,data_angular,data_momentum,i,j)
    return(final_jets_copy,data_angular,data_momentum,min_dis,indexes,indexes_jets)
def remove(indexes,indexes_jets,final_jets,data_angular,data_momentum,i):
    if (len(final_jets)==0):
        final_jets=[data_angular[i]]
    else:
        final_jets=np.concatenate((final_jets,[data_angular[i]]),0)
    indexes_jets.append(indexes[i])
    indexes_jets[len(indexes_jets)-1]=list(itertools.chain(*indexes_jets[len(indexes_jets)-1]))
    indexes.pop(i)
    return(indexes_jets,indexes,final_jets,np.delete(data_angular,i,0),np.delete(data_momentum,i,0))
def merge(indexes,data_angular,data_momentum,i,j):
    
    data_momentum[i]=data_momentum[i]+data_momentum[j]
    data_angular[i]=transform_to_angular_4(data_momentum[i])#we introduce the 4
    data_angular=np.delete(data_angular,j,0)
    data_momentum=np.delete(data_momentum,j,0)
    indexes[i].append(list(itertools.chain(*indexes[j])))
    indexes.pop(j)
    return(indexes,data_angular,data_momentum)



In [7]:
# ANTIKT, KT AND CAM/AACHEN ALGORITHM
def run_kt_inclusive(data_angular,data_momentum,a,R,qsearch,ptcut,shots,a_power):
    data_angular_copy=data_angular.copy()
    data_momentum_copy=data_momentum.copy()
    final_jets=[]
    indexes_jets=[]
    
    indexes=[]
    for i in range(len(data_angular)):
        indexes.append([[i]])
    while (len(data_momentum_copy)>1):
        final_jets,data_angular_copy,data_momentum_copy,min_dis,indexes,indexes_jets=merge_or_remove(
            indexes,indexes_jets,final_jets,data_angular_copy,
            data_momentum_copy,a,R,qsearch,ptcut,shots,a_power)
        
    a,b,final_jets,data_angular_copy,data_momentum_copy=remove(indexes,indexes_jets,final_jets,data_angular_copy,data_momentum_copy,0)
    #final_jets,indexes_jets=merge_or_remove_last(indexes,indexes_jets,final_jets,data_angular_copy,data_momentum_copy,a,R,quantum,ptcut)
    #for i in range(len(indexes_jets)):
    #    indexes_jets[i]=list(itertools.chain(*indexes_jets[i]))
    return(final_jets,data_angular_copy,data_momentum_copy,indexes_jets,indexes) 

In [8]:
points=np.loadtxt('realdata.txt')

In [9]:
#CLASSIC ANTI KT A_POWER=1
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=False
a_power=1
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_c_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_c_a1=transform_all_to_momentum_4(jets_minus2_c_a1[0])

list_minus2_c_a1=list(np.arange(len(points)))
sol_minus2_c_a1=jets_minus2_c_a1[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_c_a1)):
    for j in range(len(sol_minus2_c_a1[i])):
        list_minus2_c_a1[sol_minus2_c_a1[i][j]]=i
centers_minus2_c_a1 = np.array(list_minus2_c_a1)    
print(list_minus2_c_a1)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
#CLASSIC KT A_POWER=1
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=False
a_power=1
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_c_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_c_a1=transform_all_to_momentum_4(jets_2_c_a1[0])

list_2_c_a1=list(np.arange(len(points)))
sol_2_c_a1=jets_2_c_a1[3]

for i in range(len(final_jets_mom_2_c_a1)):
    for j in range(len(sol_2_c_a1[i])):
        list_2_c_a1[sol_2_c_a1[i][j]]=i
centers_2_c_a1 = np.array(list_2_c_a1)    
print(list_2_c_a1)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [11]:
#CLASSIC CAM/AACHEN A_POWER=1
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=False
a_power=1
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_c_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_c_a1=transform_all_to_momentum_4(jets_0_c_a1[0])

list_0_c_a1=list(np.arange(len(points)))
sol_0_c_a1=jets_0_c_a1[3]

for i in range(len(final_jets_mom_0_c_a1)):
    for j in range(len(sol_0_c_a1[i])):
        list_0_c_a1[sol_0_c_a1[i][j]]=i
centers_0_c_a1 = np.array(list_0_c_a1)    
print(list_0_c_a1)

[8, 7, 8, 6, 8, 6, 6, 8, 8, 5, 8, 8, 8, 6, 8, 4, 8, 3, 6, 8, 8, 6, 9, 8, 8, 3, 8, 8, 8, 8, 6, 8, 2, 6, 6, 1, 6, 3, 6, 8, 6, 0, 6, 0, 1, 6, 8, 6, 9, 4, 4, 6, 6, 1, 4, 6, 4, 6, 6, 8, 3, 6, 1, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [12]:
#QUANTUM ANTI KT A_POWER=1
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=True
a_power=1
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_q_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_q_a1=transform_all_to_momentum_4(jets_minus2_q_a1[0])

list_minus2_q_a1=list(np.arange(len(points)))
sol_minus2_q_a1=jets_minus2_q_a1[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_q_a1)):
    for j in range(len(sol_minus2_q_a1[i])):
        list_minus2_q_a1[sol_minus2_q_a1[i][j]]=i
centers_minus2_q_a1 = np.array(list_minus2_q_a1)    
print(list_minus2_q_a1)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 0, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [63]:
#QUANTUM KT A_POWER=1
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=True
a_power=1
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_q_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_q_a1=transform_all_to_momentum_4(jets_2_q_a1[0])

list_2_q_a1=list(np.arange(len(points)))
sol_2_q_a1=jets_2_q_a1[3]

for i in range(len(final_jets_mom_2_q_a1)):
    for j in range(len(sol_2_q_a1[i])):
        list_2_q_a1[sol_2_q_a1[i][j]]=i
centers_2_q_a1 = np.array(list_2_q_a1)    
print(list_2_q_a1)

[4, 1, 4, 6, 4, 6, 6, 4, 4, 5, 4, 4, 4, 6, 4, 6, 4, 3, 6, 4, 4, 6, 1, 4, 4, 5, 4, 4, 4, 4, 6, 4, 2, 6, 6, 6, 6, 3, 6, 4, 6, 0, 6, 0, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 3, 6, 6, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [14]:
#QUANTUM CAM/AACHEN A_POWER=1
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=True
a_power=1
shots=70
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_q_a1=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_q_a1=transform_all_to_momentum_4(jets_0_q_a1[0])

list_0_q_a1=list(np.arange(len(points)))
sol_0_q_a1=jets_0_q_a1[3]

for i in range(len(final_jets_mom_0_q_a1)):
    for j in range(len(sol_0_q_a1[i])):
        list_0_q_a1[sol_0_q_a1[i][j]]=i
centers_0_q_a1 = np.array(list_0_q_a1)    
print(list_0_q_a1)

[1, 7, 1, 4, 1, 4, 4, 1, 1, 1, 1, 1, 1, 4, 1, 0, 1, 2, 4, 1, 1, 4, 8, 1, 1, 2, 1, 1, 1, 1, 4, 1, 3, 4, 4, 6, 0, 2, 4, 1, 4, 5, 4, 5, 0, 4, 1, 4, 8, 0, 0, 4, 4, 6, 0, 4, 0, 4, 4, 1, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


In [64]:
eff_antikt_a1,centers_minus2_q_a1_corrected=final_efficiency_converged(
    centers_minus2_c_a1,centers_minus2_q_a1)
eff_kt_a1,centers_2_q_a1_corrected=final_efficiency_converged(centers_2_c_a1,centers_2_q_a1)
eff_cam_a1,centers_0_q_a1_corrected=final_efficiency_converged(centers_0_c_a1,centers_0_q_a1)
print(eff_antikt_a1,eff_kt_a1,eff_cam_a1)

0.9921875 0.9765625 0.9453125


In [35]:
#CLASSIC ANTI KT A_POWER=2
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=False
a_power=2
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_c_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_c_a2=transform_all_to_momentum_4(jets_minus2_c_a2[0])

list_minus2_c_a2=list(np.arange(len(points)))
sol_minus2_c_a2=jets_minus2_c_a2[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_c_a2)):
    for j in range(len(sol_minus2_c_a2[i])):
        list_minus2_c_a2[sol_minus2_c_a2[i][j]]=i
centers_minus2_c_a2 = np.array(list_minus2_c_a2)    
print(list_minus2_c_a2)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [36]:
#CLASSIC KT A_POWER=2
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=False
a_power=2
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_c_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_c_a2=transform_all_to_momentum_4(jets_2_c_a2[0])

list_2_c_a2=list(np.arange(len(points)))
sol_2_c_a2=jets_2_c_a2[3]

for i in range(len(final_jets_mom_2_c_a2)):
    for j in range(len(sol_2_c_a2[i])):
        list_2_c_a2[sol_2_c_a2[i][j]]=i
centers_2_c_a2 = np.array(list_2_c_a2)    
print(list_2_c_a2)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [37]:
#CLASSIC CAM/AACHEN A_POWER=2
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=False
a_power=2
shots=70
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_c_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_c_a2=transform_all_to_momentum_4(jets_0_c_a2[0])

list_0_c_a2=list(np.arange(len(points)))
sol_0_c_a2=jets_0_c_a2[3]

for i in range(len(final_jets_mom_0_c_a2)):
    for j in range(len(sol_0_c_a2[i])):
        list_0_c_a2[sol_0_c_a2[i][j]]=i
centers_0_c_a2 = np.array(list_0_c_a2)    
print(list_0_c_a2)

[8, 7, 8, 6, 8, 6, 6, 8, 8, 5, 8, 8, 8, 6, 8, 4, 8, 3, 6, 8, 8, 6, 9, 8, 8, 3, 8, 8, 8, 8, 6, 8, 2, 6, 6, 1, 6, 3, 6, 8, 6, 0, 6, 0, 1, 6, 8, 6, 9, 4, 4, 6, 6, 1, 4, 6, 4, 6, 6, 8, 3, 6, 1, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [19]:
#QUANTUM ANTI KT A_POWER=2
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=True
a_power=2
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_q_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_q_a2=transform_all_to_momentum_4(jets_minus2_q_a2[0])

list_minus2_q_a2=list(np.arange(len(points)))
sol_minus2_q_a2=jets_minus2_q_a2[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_q_a2)):
    for j in range(len(sol_minus2_q_a2[i])):
        list_minus2_q_a2[sol_minus2_q_a2[i][j]]=i
centers_minus2_q_a2 = np.array(list_minus2_q_a2)    
print(list_minus2_q_a2)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 6, 6, 9, 0, 1, 6, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
#QUANTUM KT A_POWER=2
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=True
a_power=2
shots=50
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_q_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_q_a2=transform_all_to_momentum_4(jets_2_q_a2[0])

list_2_q_a2=list(np.arange(len(points)))
sol_2_q_a2=jets_2_q_a2[3]

for i in range(len(final_jets_mom_2_q_a2)):
    for j in range(len(sol_2_q_a2[i])):
        list_2_q_a2[sol_2_q_a2[i][j]]=i
centers_2_q_a2 = np.array(list_2_q_a2)    
print(list_2_q_a2)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 6, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [24]:
#QUANTUM CAM/AACHEN A_POWER=2
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=True
a_power=2
shots=70
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_q_a2=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_q_a2=transform_all_to_momentum_4(jets_0_q_a2[0])

list_0_q_a2=list(np.arange(len(points)))
sol_0_q_a2=jets_0_q_a2[3]

for i in range(len(final_jets_mom_0_q_a2)):
    for j in range(len(sol_0_q_a2[i])):
        list_0_q_a2[sol_0_q_a2[i][j]]=i
centers_0_q_a2 = np.array(list_0_q_a2)    
print(list_0_q_a2)

[8, 4, 8, 1, 8, 1, 1, 8, 8, 0, 8, 8, 8, 1, 8, 1, 8, 5, 1, 8, 8, 1, 9, 8, 8, 5, 8, 8, 8, 8, 1, 8, 2, 1, 1, 7, 1, 5, 1, 8, 1, 6, 1, 6, 7, 1, 8, 1, 9, 1, 3, 1, 1, 7, 3, 1, 3, 1, 1, 8, 5, 1, 7, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [38]:
eff_antikt_a2,centers_minus2_q_a2_corrected=final_efficiency_converged(
    centers_minus2_c_a2,centers_minus2_q_a2)
eff_kt_a2,centers_2_q_a2_corrected=final_efficiency_converged(centers_2_c_a2,centers_2_q_a2)
eff_cam_a2,centers_0_q_a2_corrected=final_efficiency_converged(centers_0_c_a2,centers_0_q_a2)
print(eff_antikt_a2,eff_kt_a2,eff_cam_a2)

0.984375 0.9921875 0.984375


In [39]:
#CLASSIC ANTI KT A_POWER=3
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=False
a_power=3
shots=25
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_c_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_c_a3=transform_all_to_momentum_4(jets_minus2_c_a3[0])

list_minus2_c_a3=list(np.arange(len(points)))
sol_minus2_c_a3=jets_minus2_c_a3[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_c_a3)):
    for j in range(len(sol_minus2_c_a3[i])):
        list_minus2_c_a3[sol_minus2_c_a3[i][j]]=i
centers_minus2_c_a3 = np.array(list_minus2_c_a3)    
print(list_minus2_c_a3)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [40]:
#CLASSIC KT A_POWER=3
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=False
a_power=3
shots=20
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_c_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_c_a3=transform_all_to_momentum_4(jets_2_c_a3[0])

list_2_c_a3=list(np.arange(len(points)))
sol_2_c_a3=jets_2_c_a3[3]

for i in range(len(final_jets_mom_2_c_a3)):
    for j in range(len(sol_2_c_a3[i])):
        list_2_c_a3[sol_2_c_a3[i][j]]=i
centers_2_c_a3 = np.array(list_2_c_a3)    
print(list_2_c_a3)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [41]:
#CLASSIC CAM/AACHEN A_POWER=3
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=False
a_power=3
shots=40
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_c_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_c_a3=transform_all_to_momentum_4(jets_0_c_a3[0])

list_0_c_a3=list(np.arange(len(points)))
sol_0_c_a3=jets_0_c_a3[3]

for i in range(len(final_jets_mom_0_c_a3)):
    for j in range(len(sol_0_c_a3[i])):
        list_0_c_a3[sol_0_c_a3[i][j]]=i
centers_0_c_a3 = np.array(list_0_c_a3)    
print(list_0_c_a3)

[8, 7, 8, 6, 8, 6, 6, 8, 8, 5, 8, 8, 8, 6, 8, 4, 8, 3, 6, 8, 8, 6, 9, 8, 8, 3, 8, 8, 8, 8, 6, 8, 2, 6, 6, 1, 6, 3, 6, 8, 6, 0, 6, 0, 1, 6, 8, 6, 9, 4, 4, 6, 6, 1, 4, 6, 4, 6, 6, 8, 3, 6, 1, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [26]:
#QUANTUM ANTI KT A_POWER=3
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=True
a_power=3
shots=25
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_q_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_q_a3=transform_all_to_momentum_4(jets_minus2_q_a3[0])

list_minus2_q_a3=list(np.arange(len(points)))
sol_minus2_q_a3=jets_minus2_q_a3[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_q_a3)):
    for j in range(len(sol_minus2_q_a3[i])):
        list_minus2_q_a3[sol_minus2_q_a3[i][j]]=i
centers_minus2_q_a3 = np.array(list_minus2_q_a3)    
print(list_minus2_q_a3)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 6, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [27]:
#QUANTUM KT A_POWER=3
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=True
a_power=3
shots=20
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_q_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_q_a3=transform_all_to_momentum_4(jets_2_q_a3[0])

list_2_q_a3=list(np.arange(len(points)))
sol_2_q_a3=jets_2_q_a3[3]

for i in range(len(final_jets_mom_2_q_a3)):
    for j in range(len(sol_2_q_a3[i])):
        list_2_q_a3[sol_2_q_a3[i][j]]=i
centers_2_q_a3 = np.array(list_2_q_a3)    
print(list_2_q_a3)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 6, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [55]:
#QUANTUM CAM/AACHEN A_POWER=3
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=True
a_power=3
shots=40
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_q_a3=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_q_a3=transform_all_to_momentum_4(jets_0_q_a3[0])

list_0_q_a3=list(np.arange(len(points)))
sol_0_q_a3=jets_0_q_a3[3]

for i in range(len(final_jets_mom_0_q_a3)):
    for j in range(len(sol_0_q_a3[i])):
        list_0_q_a3[sol_0_q_a3[i][j]]=i
centers_0_q_a3 = np.array(list_0_q_a3)    
print(list_0_q_a3)

[1, 7, 1, 0, 1, 0, 0, 1, 1, 4, 1, 1, 1, 0, 1, 8, 1, 6, 0, 1, 1, 0, 9, 1, 1, 6, 1, 1, 1, 1, 0, 1, 3, 0, 0, 2, 0, 6, 0, 1, 0, 5, 0, 5, 2, 0, 1, 0, 9, 8, 8, 0, 0, 2, 8, 0, 8, 0, 0, 1, 6, 0, 2, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [56]:
eff_antikt_a3,centers_minus2_q_a3_corrected=final_efficiency_converged(
    centers_minus2_c_a3,centers_minus2_q_a3)
eff_kt_a3,centers_2_q_a3_corrected=final_efficiency_converged(centers_2_c_a3,centers_2_q_a3)
eff_cam_a3,centers_0_q_a3_corrected=final_efficiency_converged(centers_0_c_a3,centers_0_q_a3)
print(eff_antikt_a3,eff_kt_a3,eff_cam_a3)

0.9921875 0.9765625 1.0


In [43]:
#CLASSIC ANTI KT A_POWER=4
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=False
a_power=4
shots=15
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_c_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_c_a4=transform_all_to_momentum_4(jets_minus2_c_a4[0])

list_minus2_c_a4=list(np.arange(len(points)))
sol_minus2_c_a4=jets_minus2_c_a4[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_c_a4)):
    for j in range(len(sol_minus2_c_a4[i])):
        list_minus2_c_a4[sol_minus2_c_a4[i][j]]=i
centers_minus2_c_a4 = np.array(list_minus2_c_a4)    
print(list_minus2_c_a4)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [44]:
# CLASSIC KT A_POWER=4
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=False
a_power=4
shots=15
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_c_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_c_a4=transform_all_to_momentum_4(jets_2_c_a4[0])

list_2_c_a4=list(np.arange(len(points)))
sol_2_c_a4=jets_2_c_a4[3]

for i in range(len(final_jets_mom_2_c_a4)):
    for j in range(len(sol_2_c_a4[i])):
        list_2_c_a4[sol_2_c_a4[i][j]]=i
centers_2_c_a4 = np.array(list_2_c_a4)    
print(list_2_c_a4)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [45]:
#CLASSIC CAM/AACHEN A_POWER=4
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=False
a_power=4
shots=20
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_c_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_c_a4=transform_all_to_momentum_4(jets_0_c_a4[0])

list_0_c_a4=list(np.arange(len(points)))
sol_0_c_a4=jets_0_c_a4[3]

for i in range(len(final_jets_mom_0_c_a4)):
    for j in range(len(sol_0_c_a4[i])):
        list_0_c_a4[sol_0_c_a4[i][j]]=i
centers_0_c_a4 = np.array(list_0_c_a4)    
print(list_0_c_a4)

[8, 7, 8, 6, 8, 6, 6, 8, 8, 5, 8, 8, 8, 6, 8, 4, 8, 3, 6, 8, 8, 6, 9, 8, 8, 3, 8, 8, 8, 8, 6, 8, 2, 6, 6, 1, 6, 3, 6, 8, 6, 0, 6, 0, 1, 6, 8, 6, 9, 4, 4, 6, 6, 1, 4, 6, 4, 6, 6, 8, 3, 6, 1, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [29]:
#QUANTUM ANTI KT A_POWER=4
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=True
a_power=4
shots=15
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_q_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_q_a4=transform_all_to_momentum_4(jets_minus2_q_a4[0])

list_minus2_q_a4=list(np.arange(len(points)))
sol_minus2_q_a4=jets_minus2_q_a4[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_q_a4)):
    for j in range(len(sol_minus2_q_a4[i])):
        list_minus2_q_a4[sol_minus2_q_a4[i][j]]=i
centers_minus2_q_a4 = np.array(list_minus2_q_a4)    
print(list_minus2_q_a4)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [59]:
#QUANTUM KT A_POWER=4
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=True
a_power=4
shots=15
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_q_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_q_a4=transform_all_to_momentum_4(jets_2_q_a4[0])

list_2_q_a4=list(np.arange(len(points)))
sol_2_q_a4=jets_2_q_a4[3]

for i in range(len(final_jets_mom_2_q_a4)):
    for j in range(len(sol_2_q_a4[i])):
        list_2_q_a4[sol_2_q_a4[i][j]]=i
centers_2_q_a4 = np.array(list_2_q_a4)    
print(list_2_q_a4)

[4, 1, 4, 6, 4, 6, 6, 4, 4, 5, 4, 4, 4, 6, 4, 6, 4, 3, 6, 4, 4, 6, 1, 4, 4, 5, 4, 4, 4, 4, 6, 4, 2, 6, 6, 5, 6, 3, 6, 4, 6, 0, 6, 0, 6, 6, 4, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 4, 3, 6, 6, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [31]:
#QUANTUM CAM/AACHEN A_POWER=4
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=True
a_power=4
shots=20
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_q_a4=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_q_a4=transform_all_to_momentum_4(jets_0_q_a4[0])

list_0_q_a4=list(np.arange(len(points)))
sol_0_q_a4=jets_0_q_a4[3]

for i in range(len(final_jets_mom_0_q_a4)):
    for j in range(len(sol_0_q_a4[i])):
        list_0_q_a4[sol_0_q_a4[i][j]]=i
centers_0_q_a4 = np.array(list_0_q_a4)    
print(list_0_q_a4)

[6, 0, 6, 8, 6, 8, 8, 6, 6, 4, 6, 6, 6, 8, 6, 2, 6, 3, 8, 6, 6, 8, 9, 6, 6, 3, 6, 6, 6, 6, 8, 6, 5, 8, 8, 7, 8, 3, 8, 6, 8, 1, 8, 1, 7, 8, 6, 8, 9, 2, 2, 8, 8, 7, 2, 8, 2, 8, 8, 6, 3, 8, 7, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [60]:
eff_antikt_a4,centers_minus2_q_a4_corrected=final_efficiency_converged(
    centers_minus2_c_a4,centers_minus2_q_a4)
eff_kt_a4,centers_2_q_a4_corrected=final_efficiency_converged(centers_2_c_a4,centers_2_q_a4)
eff_cam_a4,centers_0_q_a4_corrected=final_efficiency_converged(centers_0_c_a4,centers_0_q_a4)
print(eff_antikt_a4,eff_kt_a4,eff_cam_a4)

1.0 1.0 1.0


In [47]:
#CLASSIC ANTI KT A_POWER=5
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=False
a_power=5
shots=5
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_c_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_c_a5=transform_all_to_momentum_4(jets_minus2_c_a5[0])

list_minus2_c_a5=list(np.arange(len(points)))
sol_minus2_c_a5=jets_minus2_c_a5[3]

for i in range(len(final_jets_mom_minus2_c_a5)):
    for j in range(len(sol_minus2_c_a5[i])):
        list_minus2_c_a5[sol_minus2_c_a5[i][j]]=i
centers_minus2_c_a5 = np.array(list_minus2_c_a5)    
print(list_minus2_c_a5)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 0, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [48]:
#CLASSIC KT A_POWER=5
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=False
a_power=5
shots=8
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_c_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_c_a5=transform_all_to_momentum_4(jets_2_c_a5[0])

list_2_c_a5=list(np.arange(len(points)))
sol_2_c_a5=jets_2_c_a5[3]

for i in range(len(final_jets_mom_2_c_a5)):
    for j in range(len(sol_2_c_a5[i])):
        list_2_c_a5[sol_2_c_a5[i][j]]=i
centers_2_c_a5 = np.array(list_2_c_a5)    
print(list_2_c_a5)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 5, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 5, 6, 6, 6, 6, 5, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [49]:
#CLASSIC CAM/AACHEN A_POWER=5
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=False
a_power=5
shots=10
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_c_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_c_a5=transform_all_to_momentum_4(jets_0_c_a5[0])

list_0_c_a5=list(np.arange(len(points)))
sol_0_c_a5=jets_0_c_a5[3]

for i in range(len(final_jets_mom_0_c_a5)):
    for j in range(len(sol_0_c_a5[i])):
        list_0_c_a5[sol_0_c_a5[i][j]]=i
centers_0_c_a5 = np.array(list_0_c_a5)    
print(list_0_c_a5)

[8, 7, 8, 6, 8, 6, 6, 8, 8, 5, 8, 8, 8, 6, 8, 4, 8, 3, 6, 8, 8, 6, 9, 8, 8, 3, 8, 8, 8, 8, 6, 8, 2, 6, 6, 1, 6, 3, 6, 8, 6, 0, 6, 0, 1, 6, 8, 6, 9, 4, 4, 6, 6, 1, 4, 6, 4, 6, 6, 8, 3, 6, 1, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [51]:
#QUANTUM ANTI KT A_POWER=5
a=-2 #antikt
R=1
ptcut=10 #GeV
qsearch=True
a_power=5
shots=5
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_minus2_q_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_minus2_q_a5=transform_all_to_momentum_4(jets_minus2_q_a5[0])

list_minus2_q_a5=list(np.arange(len(points)))
sol_minus2_q_a5=jets_minus2_q_a5[3]
#print(sol[1].index(13))
for i in range(len(final_jets_mom_minus2_q_a5)):
    for j in range(len(sol_minus2_q_a5[i])):
        list_minus2_q_a5[sol_minus2_q_a5[i][j]]=i
centers_minus2_q_a5 = np.array(list_minus2_q_a5)    
print(list_minus2_q_a5)

[3, 5, 3, 0, 3, 0, 0, 2, 3, 1, 3, 8, 3, 0, 3, 6, 3, 2, 0, 3, 3, 0, 9, 3, 3, 1, 8, 3, 3, 7, 0, 3, 4, 0, 0, 1, 0, 2, 0, 3, 0, 7, 0, 7, 6, 0, 9, 0, 9, 6, 6, 9, 0, 1, 4, 0, 6, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [33]:
#QUANTUM KT A_POWER=5
a=2 #kt
R=1
ptcut=10 #GeV
qsearch=True
a_power=5
shots=8
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_2_q_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_2_q_a5=transform_all_to_momentum_4(jets_2_q_a5[0])

list_2_q_a5=list(np.arange(len(points)))
sol_2_q_a5=jets_2_q_a5[3]

for i in range(len(final_jets_mom_2_q_a5)):
    for j in range(len(sol_2_q_a5[i])):
        list_2_q_a5[sol_2_q_a5[i][j]]=i
centers_2_q_a5 = np.array(list_2_q_a5)    
print(list_2_q_a5)

[3, 1, 3, 6, 3, 6, 6, 3, 3, 5, 3, 3, 3, 6, 3, 6, 3, 4, 6, 3, 3, 6, 1, 3, 3, 5, 3, 3, 3, 3, 6, 3, 2, 6, 6, 6, 6, 4, 6, 3, 6, 0, 6, 0, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 4, 6, 6, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [34]:
#QUANTUM CAM/AACHEN A_POWER=5
a=0 #Cam
R=1
ptcut=10 #GeV
qsearch=True
a_power=5
shots=10
points_energy=points
points_angular=transform_all_to_angular_4(points_energy)

jets_0_q_a5=run_kt_inclusive(points_angular,points_energy,a,R,qsearch,ptcut,shots,a_power)
final_jets_mom_0_q_a5=transform_all_to_momentum_4(jets_0_q_a5[0])

list_0_q_a5=list(np.arange(len(points)))
sol_0_q_a5=jets_0_q_a5[3]

for i in range(len(final_jets_mom_0_q_a5)):
    for j in range(len(sol_0_q_a5[i])):
        list_0_q_a5[sol_0_q_a5[i][j]]=i
centers_0_q_a5 = np.array(list_0_q_a5)    
print(list_0_q_a5)

[6, 2, 6, 8, 6, 8, 8, 6, 6, 0, 6, 6, 6, 8, 6, 1, 6, 7, 8, 6, 6, 8, 9, 6, 6, 7, 6, 6, 6, 6, 8, 6, 3, 8, 8, 4, 8, 7, 8, 6, 8, 5, 8, 5, 4, 8, 6, 8, 9, 1, 1, 8, 8, 4, 1, 8, 1, 8, 8, 6, 7, 8, 4, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [52]:
eff_antikt_a5,centers_minus2_q_a5_corrected=final_efficiency_converged(
    centers_minus2_c_a5,centers_minus2_q_a5)
eff_kt_a5,centers_2_q_a5_corrected=final_efficiency_converged(centers_2_c_a5,centers_2_q_a5)
eff_cam_a5,centers_0_q_a5_corrected=final_efficiency_converged(centers_0_c_a5,centers_0_q_a5)
print(eff_antikt_a5,eff_kt_a5,eff_cam_a5)

0.9921875 0.9765625 1.0
